In [1]:
! pip install -Uq sentence-transformers transformers datasets faiss-gpu-cu12

### **Model**

In [2]:
import torch
from sentence_transformers import SentenceTransformer

# Download from the 🤗 Hub
model = SentenceTransformer(
    "rasyosef/roberta-amharic-text-embedding-base",
    device="cuda" if torch.cuda.is_available() else "cpu"
  )

# Run inference
sentences = [
  "የተደጋገመው የመሬት መንቀጥቀጥና የእሳተ ገሞራ ምልክት በአፋር ክልል",
  "ከተደጋጋሚ መሬት መንቀጥቀጥ በኋላ አፋር ክልል እሳት ከመሬት ውስጥ ሲፈላ ታይቷል፡፡ ከመሬት ውስጥ እሳትና ጭስ የሚተፋው እንፋሎቱ ዛሬ ማለዳውን 11 ሰዓት ግድም ከከባድ ፍንዳታ በኋላየተስተዋለ መሆኑን የአከባቢው ነዋሪዎች እና ባለስልጣናት ለዶቼ ቬለ ተናግረዋል፡፡ አለት የሚያፈናጥር እሳት ነው የተባለው እንፋሎቱ በክልሉ ጋቢረሱ (ዞን 03) ዱለቻ ወረዳ ሰጋንቶ ቀበሌ መከሰቱን የገለጹት የአከባቢው የአይን እማኞች ከዋናው ፍንዳታ በተጨማሪ በዙሪያው ተጨማሪ ፍንዳታዎች መታየት ቀጥሏል ባይ ናቸው፡፡",
  "ለኢትዮጵያ ብሔራዊ ባንክ ዋጋን የማረጋጋት ቀዳሚ ዓላማ ጋር የተጣጣሙ የገንዘብ ፖሊሲ ምክረ ሀሳቦችን እንዲሰጥ የተቋቋመው የኢትዮጵያ ብሔራዊ ባንክ የገንዘብ ፖሊሲ ኮሚቴ እስካለፈው ህዳር ወር የነበረው እአአ የ2024 የዋጋ ግሽበት በተለይምምግብ ነክ ምርቶች ላይ ከአንድ ዓመት በፊት ከነበው ጋር ሲነጻጸር መረጋጋት ማሳየቱን ጠቁሟል፡፡ ዶይቼ ቬለ ያነጋገራቸው የአዲስ አበባ ነዋሪዎች ግን በዚህ የሚስማሙ አይመስልም፡፡ ከአምና አንጻር ያልጨመረ ነገር የለም ባይ ናቸው፡፡ የኢኮኖሚ  ባለሙያም በሰጡን አስተያየት ጭማሪው በሁሉም ረገድ የተስተዋለ በመሆኑ የመንግስት ወጪን በመቀነስ ግብርናው ላይ አተኩሮ መስራት ምናልባትም የዋጋ መረጋጋቱን ሊያመጣ ይችላል ይላሉ፡፡"
]
embeddings = model.encode(sentences)
print(embeddings.shape)
# [3, 512]

# Get the similarity scores for the embeddings
similarities = model.similarity(embeddings, embeddings)
print(similarities)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


(3, 768)
tensor([[ 1.0000,  0.7002,  0.0047],
        [ 0.7002,  1.0000, -0.0130],
        [ 0.0047, -0.0130,  1.0000]])


### **Dataset**

In [3]:
from datasets import load_dataset

dataset = load_dataset("rasyosef/amharic-news-retrieval-dataset", split="train")
dataset

Dataset({
    features: ['query_id', 'passage_id', 'query', 'passage', 'category', 'link'],
    num_rows: 44708
})

In [4]:
model.encode(["የተደጋገመው የመሬት መንቀጥቀጥና የእሳተ ገሞራ ምልክት በአፋር ክልል"]).shape

(1, 768)

In [ ]:
# convert embeddings to numpy arrays for use with FAISS
embeddings_dataset = dataset.map(
    lambda x: {"embeddings": model.encode(x["passage"])},
    batched=True,
    batch_size=256
)

embeddings_dataset

##### **Using FAISS Index for similarity search**

In [ ]:
# create faiss index for the embeddings dataset
import faiss

embeddings_dataset.add_faiss_index(column="embeddings", metric_type=faiss.METRIC_INNER_PRODUCT)

### **Search**

- **Query 1:** "የለንደን ማራቶን ሊሰረዝ ይችላል"
  - **Top Document:** "በቀጣዩ መስከረም ሊካሄድ ቀጠሮ የተያዘለት ‹‹የግሬት ኖርዝ ረን›› ግማሽ ማራቶን ውድድር መሰረዙን ተከትሎ ጥቅምት ላይ እንደሚካሄድ የተነገረው የለንደን ማራቶንም ሊሰረዝ እንደሚችል ጥርጣሬ ፈጥሯል..."
  - **Similarity (Relevance) Score:** `0.7415`
- **Query 2:** "የለንደን ማራቶን አልተሰረዘም"
  - **Top Document:** "በቀጣዩ መስከረም ሊካሄድ ቀጠሮ የተያዘለት ‹‹የግሬት ኖርዝ ረን›› ግማሽ ማራቶን ውድድር መሰረዙን ተከትሎ ጥቅምት ላይ እንደሚካሄድ የተነገረው የለንደን ማራቶንም ሊሰረዝ እንደሚችል ጥርጣሬ ፈጥሯል..."
  - **Similarity (Relevance) Score:** `0.729`



##### **Query 1**

In [15]:
# nearest neighbor lookup on faiss index

query = "የለንደን ማራቶን ሊሰረዝ ይችላል" #"ደቡብ ሱዳን በጦርነት አፋፍ ስጋት ላይ ነች" #"በ2023 አመት አዲስ አበባ ውስጥ ያልከናወኑ ዝግጅቶች"
query_embedding = model.encode(query)
query_embedding.shape

(768,)

In [16]:
scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", query_embedding, k=5
)

for score, candidate_id, candidate_text in zip(scores[1:], samples["passage_id"], samples["passage"]):
  print("SIMILARITY SCORE:", score)
  print("CANDIDATE ID:", candidate_id)
  print("CANDIDATE TEXT:", candidate_text)
  print()

SIMILARITY SCORE: 0.7415061
CANDIDATE ID: ed1349c7a3a3792606c14c89ef1f693d
CANDIDATE TEXT: በቀጣዩ መስከረም ሊካሄድ ቀጠሮ የተያዘለት ‹‹የግሬት ኖርዝ ረን›› ግማሽ ማራቶን ውድድር መሰረዙን ተከትሎ ጥቅምት ላይ እንደሚካሄድ የተነገረው የለንደን ማራቶንም ሊሰረዝ እንደሚችል ጥርጣሬ ፈጥሯል:: በእንግሊዝ አገር ከሚካሄዱ የጎዳና ላይ ውድድሮች መካከል ትልቁን ስፍራ የሚይዙት ሁለቱ የማራቶንና ግማሽ ማራቶን ውድድሮች ከኮሮና ቫይረስ(ኮቪድ 19) ወረርሽኝ ስጋት ጋር በተያያዘ ቀደም ብሎ ከሚካሄዱበት ወቅት እንዲራዘሙ ተደርጎ በቀጣዩ መስከረምና ጥቅምት ወር ሊካሄዱ ቀጠሮ ተይዞ ነበር፡፡ 
ዘንድሮ አርባኛ ዓመቱን የያዘው ‹‹ግሬት ኖርዝ ረን›› የግማሽ ማራቶን ውድድር ከመሰረዙ አስቀድሞ በታሪክ ለመጀመሪያ ጊዜ ስልሳ ሺ ተሳታፊዎችን በመያዝ በብሪታኒያ ትልቁ የጎዳና ላይ ውድድር መሆን ችሏል፡፡ ከሰላሳ አንድ ሚሊዮን ዶላር በላይ ከውድድሩ በመሰብሰብ ለበጎ አድራጎት እንዲውል ለማድረግ ታስቦ የነበረ ቢሆንም በኮሮና ቫይረስ ወረርሽኝ ሳቢያ እንዳልተቻለ አዘጋጆቹ ገልፀዋል፡፡ ይህንንም ተከትሎ ውድድሩ ሊሰረዝ እንደቻለ የሮይተርስ ዘገባ ያመለክታል:: ውድድሩ ለኮሮና ቫይረስ ወረርሽኝ የሚደረገውን ጥንቃቄ ከግምት በማስገባት እንዲካሄድ አዘጋጆቹ ከጤና ባለሙያዎች እንዲሁም ከሚመለከታቸው አካላት ጋር ተቀናጅተው በመስራት ጥረት ቢያደርጉም እንዳልተሳካ አሳውቀዋል፡፡ 
በዚህ ታላቅ ውድድር ባለፈው ዓመት እንግሊዛዊው
 የኦሊምፒክ ጥምር የወርቅ ሜዳሊያ ባለቤት የሆነው አትሌት ሞሐመድ ፋራህ ለስድስተኛ ተከታታይ ጊዜ ሲያሸንፍ ኬንያዊቷ የማራቶን ባለክብረወሰን ብሪጊድ ኮስጌ የውድድሩን ክብረወሰን በማሻሻል ጭምር ማሸነፏ ይታወሳል፡፡ 
ይህ የግማ

##### **Query 2**

In [17]:
# nearest neighbor lookup on faiss index

query = "የለንደን ማራቶን አልተሰረዘም" #"የለንደን ማራቶን ሊሰረዝ ይችላል" #"ደቡብ ሱዳን በጦርነት አፋፍ ስጋት ላይ ነች" #"በ2023 አመት አዲስ አበባ ውስጥ ያልከናወኑ ዝግጅቶች"
query_embedding = model.encode(query)
query_embedding.shape

(768,)

In [18]:
scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", query_embedding, k=5
)

for score, candidate_id, candidate_text in zip(scores[1:], samples["passage_id"], samples["passage"]):
  print("SIMILARITY SCORE:", score)
  print("CANDIDATE ID:", candidate_id)
  print("CANDIDATE TEXT:", candidate_text)
  print()

SIMILARITY SCORE: 0.7293599
CANDIDATE ID: ed1349c7a3a3792606c14c89ef1f693d
CANDIDATE TEXT: በቀጣዩ መስከረም ሊካሄድ ቀጠሮ የተያዘለት ‹‹የግሬት ኖርዝ ረን›› ግማሽ ማራቶን ውድድር መሰረዙን ተከትሎ ጥቅምት ላይ እንደሚካሄድ የተነገረው የለንደን ማራቶንም ሊሰረዝ እንደሚችል ጥርጣሬ ፈጥሯል:: በእንግሊዝ አገር ከሚካሄዱ የጎዳና ላይ ውድድሮች መካከል ትልቁን ስፍራ የሚይዙት ሁለቱ የማራቶንና ግማሽ ማራቶን ውድድሮች ከኮሮና ቫይረስ(ኮቪድ 19) ወረርሽኝ ስጋት ጋር በተያያዘ ቀደም ብሎ ከሚካሄዱበት ወቅት እንዲራዘሙ ተደርጎ በቀጣዩ መስከረምና ጥቅምት ወር ሊካሄዱ ቀጠሮ ተይዞ ነበር፡፡ 
ዘንድሮ አርባኛ ዓመቱን የያዘው ‹‹ግሬት ኖርዝ ረን›› የግማሽ ማራቶን ውድድር ከመሰረዙ አስቀድሞ በታሪክ ለመጀመሪያ ጊዜ ስልሳ ሺ ተሳታፊዎችን በመያዝ በብሪታኒያ ትልቁ የጎዳና ላይ ውድድር መሆን ችሏል፡፡ ከሰላሳ አንድ ሚሊዮን ዶላር በላይ ከውድድሩ በመሰብሰብ ለበጎ አድራጎት እንዲውል ለማድረግ ታስቦ የነበረ ቢሆንም በኮሮና ቫይረስ ወረርሽኝ ሳቢያ እንዳልተቻለ አዘጋጆቹ ገልፀዋል፡፡ ይህንንም ተከትሎ ውድድሩ ሊሰረዝ እንደቻለ የሮይተርስ ዘገባ ያመለክታል:: ውድድሩ ለኮሮና ቫይረስ ወረርሽኝ የሚደረገውን ጥንቃቄ ከግምት በማስገባት እንዲካሄድ አዘጋጆቹ ከጤና ባለሙያዎች እንዲሁም ከሚመለከታቸው አካላት ጋር ተቀናጅተው በመስራት ጥረት ቢያደርጉም እንዳልተሳካ አሳውቀዋል፡፡ 
በዚህ ታላቅ ውድድር ባለፈው ዓመት እንግሊዛዊው
 የኦሊምፒክ ጥምር የወርቅ ሜዳሊያ ባለቤት የሆነው አትሌት ሞሐመድ ፋራህ ለስድስተኛ ተከታታይ ጊዜ ሲያሸንፍ ኬንያዊቷ የማራቶን ባለክብረወሰን ብሪጊድ ኮስጌ የውድድሩን ክብረወሰን በማሻሻል ጭምር ማሸነፏ ይታወሳል፡፡ 
ይህ የግማ